In [1]:
import io
import math
import keras
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

from google.colab import files

plt.style.use('default')

In [10]:
# https://finance.yahoo.com/quote/AAPL/history?period1=345427200&period2=1624233600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true (Max Historical Apple Daily)
resp = requests.get(f'https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=345427200&period2=1624233600&interval=1d&events=history&includeAdjustedClose=true') # download link
csv_data = io.StringIO(resp.content.decode('utf8').replace("'", '"'))
df = pd.read_csv(csv_data, sep=',')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095495,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088485,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090676,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093304,73449600
...,...,...,...,...,...,...,...
10211,2021-06-14,127.820000,130.539993,127.070000,130.479996,130.479996,96906500
10212,2021-06-15,129.940002,130.600006,129.389999,129.639999,129.639999,62746300
10213,2021-06-16,130.369995,130.889999,128.460007,130.149994,130.149994,91815000
10214,2021-06-17,129.800003,132.550003,129.649994,131.789993,131.789993,96721700


In [11]:
df['EMA12'] = df['Adj Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Adj Close'].ewm(span=26, adjust=False).mean()

df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean() 

df['Next Adj Close'] = df['Adj Close'].shift(-1)
df.dropna(inplace=True)
df

,Date,Open,High,Low,Close,Adj Close,Volume,EMA12,EMA26,MACD,Signal,Next Adj Close
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600,0.100751,0.100751,0.000000,0.000000,0.095495
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095495,175884800,0.099942,0.100362,-0.000419,-0.000084,0.088485
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088485,105728000,0.098180,0.099482,-0.001302,-0.000328,0.090676
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090676,86441600,0.097025,0.098830,-0.001804,-0.000623,0.093304
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093304,73449600,0.096453,0.098420,-0.001968,-0.000892,0.098999
...,...,...,...,...,...,...,...,...,...,...,...,...
10210,2021-06-11,126.529999,127.440002,126.099998,127.349998,127.349998,53522400,126.201356,126.598074,-0.396718,-0.720339,130.479996
10211,2021-06-14,127.820000,130.539993,127.070000,130.479996,130.479996,96906500,126.859608,126.885624,-0.026015,-0.581474,129.639999
10212,2021-06-15,129.940002,130.600006,129.389999,129.639999,129.639999,62746300,127.287361,127.089651,0.197709,-0.425637,130.149994
10213,2021-06-16,130.369995,130.889999,128.460007,130.149994,130.149994,91815000,127.727766,127.316344,0.411422,-0.258225,131.789993


In [12]:
X = df[['Open','High','Low','Adj Close','Volume','EMA12','EMA26','MACD','Signal']]
y = df[['Next Adj Close']]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, stratify=None)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7661, 9), (2554, 9), (7661, 1), (2554, 1))

In [13]:
X_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = X_scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_scaled, columns=[X_train.columns])

print ('X_train inputs:', X_train.columns)

window = 1 # 1 day

X_train_window=[]
y_train_window=[]
for i in range(window, len(X_train)):
  X_train_window.append(X_scaled[i-window:i, :])
  y_train_window.append(X_scaled[i, 0])

X_train_window, y_train_window = np.array(X_train_window), np.array(y_train_window)
X_train_window = np.reshape(X_train_window, (X_train_window.shape[0], X_train_window.shape[1], len(X_train.columns)))

print ('X_train_window.shape', X_train_window.shape)
print ('y_train_window.shape', y_train_window.shape)

X_train inputs: MultiIndex([(     'Open',),
            (     'High',),
            (      'Low',),
            ('Adj Close',),
            (   'Volume',),
            (    'EMA12',),
            (    'EMA26',),
            (     'MACD',),
            (   'Signal',)],
           )
X_train_window.shape (7660, 1, 9)
y_train_window.shape (7660,)


In [ ]:
def create_model():
  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_window.shape[1], len(X_train.columns)))) # layer 1 lstm
  model.add(Dropout(0.2)) # layer 1 dropout regularisation
  model.add(LSTM(units=50, return_sequences=True)) # layer 2 lstm
  model.add(Dropout(0.2)) # layer 2 dropout regularisation
  model.add(LSTM(units=50, return_sequences=True)) # layer 3 lstm
  model.add(Dropout(0.2)) # layer 3 dropout regularisation
  model.add(LSTM(units=50)) # layer 4 lstm
  model.add(Dropout(0.2)) # layer 4 dropout regularisation
  model.add(Dense(units=1)) # output layer
  model.compile(optimizer='adam', loss='mean_squared_error') # compile the rnn 
  return model

model = create_model()
model.fit(X_train_window, y_train_window, epochs=500, batch_size=32, verbose=2)
model.summary()

Epoch 1/500
240/240 - 6s - loss: 0.0077
Epoch 2/500
240/240 - 1s - loss: 9.8203e-04
Epoch 3/500
240/240 - 1s - loss: 8.7737e-04
Epoch 4/500
240/240 - 1s - loss: 8.3598e-04
Epoch 5/500
240/240 - 1s - loss: 7.2610e-04
Epoch 6/500
240/240 - 1s - loss: 6.4470e-04
Epoch 7/500
240/240 - 1s - loss: 6.7813e-04
Epoch 8/500
240/240 - 1s - loss: 6.7870e-04
Epoch 9/500
240/240 - 1s - loss: 6.6384e-04
Epoch 10/500
240/240 - 1s - loss: 6.7087e-04
Epoch 11/500
240/240 - 1s - loss: 6.5120e-04
Epoch 12/500
240/240 - 1s - loss: 6.3521e-04
Epoch 13/500
240/240 - 1s - loss: 5.5656e-04
Epoch 14/500
240/240 - 1s - loss: 5.6446e-04
Epoch 15/500
240/240 - 1s - loss: 6.1662e-04
Epoch 16/500
240/240 - 1s - loss: 5.9824e-04
Epoch 17/500
240/240 - 1s - loss: 6.3317e-04
Epoch 18/500
240/240 - 1s - loss: 5.1245e-04
Epoch 19/500
240/240 - 1s - loss: 6.1560e-04
Epoch 20/500
240/240 - 1s - loss: 5.9707e-04
Epoch 21/500
240/240 - 1s - loss: 6.5154e-04
Epoch 22/500
240/240 - 1s - loss: 5.3091e-04
Epoch 23/500
240/240 - 

In [ ]:
'''Optionally save model, model.json and weights.h5'''

model_filename = f'model_apple_daily_{len(X_train.columns)}-inputs.json'
weights_filename = f'weights_apple_daily_{len(X_train.columns)}-inputs.h5'

!ls /content

# save structure to json
model_json = model.to_json()
with open(model_filename, 'w') as json_file:
  json_file.write(model_json)

# save weights to hdf5
model.save_weights(weights_filename)

files.download(f'/content/{model_filename}')
files.download(f'/content/{weights_filename}')

 model_apple_daily_12-inputs.json       weights_apple_daily_12-inputs.h5
'model_apple_daily_5-inputs (1).json'  'weights_apple_daily_5-inputs (1).h5'
 model_apple_daily_5-inputs.json        weights_apple_daily_5-inputs.h5
 sample_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''Optionally load model, model.json and weights.h5'''

model_filename = f'model_apple_daily_{len(X_train.columns)}-inputs.json'
weights_filename = f'weights_apple_daily_{len(X_train.columns)}-inputs.h5'

try:
  files.upload()
  !ls /content

  # read structure from json
  model = open(model_filename, 'r')
  json = model.read()
  model.close()
  model = model_from_json(json)

  # read weights from hdf5
  model.load_weights(f'/content/{weights_filename}')
except Exception as e:
  print (e)

Saving model_apple_daily_5-inputs.json to model_apple_daily_5-inputs (1).json
Saving weights_apple_daily_5-inputs.h5 to weights_apple_daily_5-inputs (1).h5
'model_apple_daily_5-inputs (1).json'  'weights_apple_daily_5-inputs (1).h5'
 model_apple_daily_5-inputs.json        weights_apple_daily_5-inputs.h5
 sample_data


In [ ]:
X_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = X_scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_scaled, columns=[X_test.columns])

print ('X_test inputs:', X_test.columns)

y_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaled = y_scaler.fit_transform(y)
y_test = pd.DataFrame(y_scaled, columns=[y.columns])

print ('y_test inputs:', y_test.columns[0])

X_test_window=[]
y_test_window=[]
for i in range(window, len(X_test)):
  X_test_window.append(X_scaled[i-window:i, :])
  y_test_window.append(X_scaled[i, 0])

X_test_window, y_test_window = np.array(X_test_window), np.array(y_test_window)
X_test_window = np.reshape(X_test_window, (X_test_window.shape[0], X_test_window.shape[1], len(X_test.columns)))

print ('X_test_window.shape:', X_test_window.shape)
print ('y_test_window.shape:', y_test_window.shape)

y_pred = model.predict(X_test_window)
y_pred = y_scaler.inverse_transform(y_pred)

print ('y_pred.shape:', y_pred.shape)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(df['Date'].tail(len(y_pred)), df['Adj Close'][-len(y_pred):].values, color='red', label=f'Actual Apple Daily {y.columns.values[0].title()}')
plt.plot(df['Date'].tail(len(y_pred)), y_pred, color='blue', label=f'Predicted Apple Daily {y.columns.values[0].title()}')
plt.title(f'Apple Daily {y.columns.values[0].title()} Prediction')
plt.xlabel('Time')
plt.ylabel(f'{y.columns.values[0].title()}')
plt.xticks(rotation=90)
plt.legend()
plt.show()